In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import string

In [3]:
def editdist(s1, s2):
    c = len(s1) + 1
    r = len(s2) + 1
    # initialize matrix
    M = [[0 for x in range(c)] for y in range(r)]
    for i in range(0,c):
        M[0][i] = i
    for j in range(0,r):
        M[j][0] = j
    # dynamic programming
    for i in range(1,r):
        for j in range(1,c):
            if s1[j-1] == s2[i-1]:
                M[i][j] = M[i-1][j-1]
            else:
                M[i][j] = min(M[i-1][j-1], M[i-1][j], M[i][j-1]) + 1

    
    return M[r-1][c-1]

In [4]:
fodors_raw = list()
with open("fodors.csv", "r") as f:
    i = 0
    for line in f.readlines():
        fodors_raw.append(line.split(","))
        
        if len(line.split(",")) != 3:
            print str(i) + "---" + line
        i += 1


15---"Canter's 419 N. Fairfax Ave. Los Angeles", "213/651-2030". American"

23---"Dive! 10250 Santa Monica Blvd. Los Angeles", "310/788-DIVE American"

229---"Lucky Strike 59 Grand St.  between Wooster St. and W. Broadway New York", "212/941-0479", "or", "212/941-0772", "American"

277---"Rainbow Room 30 Rockefeller Plaza New York", "212/632-5000", "or", "212/632-5100", "American"

333---"Zen Palate 34 Union Sq. E at 16th St. New York", "212/614-9291", "and", "212/614-9345", "Asian"

506---"R&G Lounge 631 B Kearny St. San Francisco", "415/982-7877", "or", "415/982-3811", "Asian"



In [5]:
# clean data
fodors_raw[15] = [fodors_raw[15][0], fodors_raw[15][1].split(". ")[0], "\"" + fodors_raw[15][1].split(". ")[1]]
fodors_raw[23] = [fodors_raw[23][0], fodors_raw[23][1].split("DIVE ")[0]+ "3483\"", "\"" + fodors_raw[23][1].split("DIVE ")[1]]
tmp = len(fodors_raw) - 1
for i in [229, 277, 333,506]:
    fodors_raw.append([fodors_raw[i][0],fodors_raw[i][3],fodors_raw[i][4]])
    fodors_raw[i] = [fodors_raw[i][0],fodors_raw[i][1],fodors_raw[i][4]]

mfodors = {tmp : 228, tmp +1 : 276, tmp + 2 : 332, tmp + 3 : 505}

In [6]:
zagats_raw = list()
with open("zagats.csv", "r") as f1:
    i=0
    for line in f1.readlines():
        zagats_raw.append(line.split(","))
        if len(line.split(",")) != 3:
            print str(i) + "---" + line
        i += 1


236---"Deacon Burton's Soulfood Restaurant 1029 Edgewood Ave. SE Atlanta404-523-1929", "Southern/Soul"

245---"Hedgerose Heights Inn  The 490 E. Paces Ferry Rd. NE Atlanta", "404-233-7673Continental"

276---"Toulouse", "293-B Peachtree Rd. Atlanta", "404-351-9533", "French (New)"



In [7]:
zagats_raw[236] = [zagats_raw[236][0][:-13], zagats_raw[236][0][-13:-1], zagats_raw[236][1]]
zagats_raw[245] = [zagats_raw[245][0], zagats_raw[245][1][0:14], zagats_raw[245][1][14:]]
zagats_raw[276] = [zagats_raw[276][0]+ zagats_raw[276][1], zagats_raw[276][2], zagats_raw[276][3]]


In [8]:
# remove header
fodors = fodors_raw[1:]
zagats = zagats_raw[1:]
len(fodors)

537

In [10]:
# initialize distance matrix
c = len(zagats)
r = len(fodors)
# addrDist = [[0 for x in range(c)] for y in range(r)]  # 533 * 331
telDist = np.zeros((len(fodors), len(zagats)))
# compare input and get edit distance
for i in range(0, r):
    for j in range(0, c):
#         addrDist[i][j] += editdist(fodors[i][0], zagats[j][0])
        telDist[i][j] += editdist(fodors[i][1].translate(None, string.punctuation), zagats[j][1].translate(None, string.punctuation))
#         cuiDist[i][j] += editdist(fodors[i][2], zagats[j][2])

In [12]:
np.histogram(telDist)

(array([  120,    47,   366,  1875,  7205, 55108, 50483, 49118, 13196,   229]),
 array([  0. ,   1.2,   2.4,   3.6,   4.8,   6. ,   7.2,   8.4,   9.6,
         10.8,  12. ]))

In [13]:
unique, counts = np.unique(telDist, return_counts=True)
dict(zip(unique, counts))

{0.0: 111,
 1.0: 9,
 2.0: 47,
 3.0: 366,
 4.0: 1875,
 5.0: 7205,
 6.0: 19413,
 7.0: 35695,
 8.0: 50483,
 9.0: 49118,
 10.0: 13196,
 11.0: 226,
 12.0: 3}

In [14]:
cand = np.where(telDist <=1)

In [15]:
cand

(array([  1,   2,  10,  13,  18,  19,  29,  32,  33,  36,  42,  43,  46,
         48,  56,  58,  59,  66,  67,  71,  75,  79,  82,  83,  94,  98,
        116, 119, 131, 132, 135, 145, 146, 158, 170, 182, 183, 197, 201,
        208, 209, 214, 223, 225, 229, 234, 236, 242, 243, 247, 253, 262,
        268, 269, 270, 276, 280, 281, 287, 292, 294, 296, 298, 299, 300,
        309, 314, 322, 327, 335, 340, 344, 345, 346, 347, 347, 351, 357,
        358, 358, 361, 362, 363, 368, 370, 376, 380, 382, 386, 386, 396,
        397, 403, 412, 413, 414, 417, 420, 429, 431, 437, 438, 444, 447,
        450, 454, 458, 460, 468, 479, 483, 486, 489, 492, 496, 503, 504,
        506, 507, 534]),
 array([  1,   2,  15,  17,  20,  21,  27,  28,  29,   5,  45,  50,  52,
         56,  67,  68,  71,  41,  75,  81,  65,  87,  89,  91,  93,  95,
         98,  99, 100, 101, 102, 106, 108, 112, 113, 115, 116, 119, 121,
        127, 129, 131, 133, 134, 136, 137, 138, 141, 142, 145, 148, 153,
        156, 157, 158, 159

In [16]:
# map back the indexes
for i in cand[0][cand[0]>= tmp]:
    cand[0][cand[0]>= tmp] = mfodors[i]

In [17]:
# calculate edit dist for addr and cuisine
addrDist = []
cuiDist = []
for i in range(len(cand[0])):
    str1 = fodors[cand[0][i]][0].translate(None, string.punctuation)
    str2 = zagats[cand[1][i]][0].translate(None, string.punctuation)
    
    add = editdist(str1, str2) *1.0 / min(len(str1),len(str2))
    
    addrDist.append(add)
    if add >= 0.95: 
        print "Big dist in addr--------"
        print str1
        print str2
    
    cstr1 = fodors[cand[0][i]][2]
    cstr2 = zagats[cand[1][i]][2]
    cui = editdist(cstr1, cstr2)
    if cui >= 21:
        print "Big dist in cui--------"
        print str1
        print str2
    cuiDist.append(cui)

Big dist in addr--------
Spago 1114 Horn Ave Los Angeles
Spago Los Angeles 8795 Sunset Blvd W Hollywood
Big dist in addr--------
Roettelle A G 126 E 7th St  between 1st Ave and Ave A New York
Kiev 117 Second Ave New York City
Big dist in addr--------
Sfuzzi 58 W 65th St New York
Tavern on the Green Central Park West New York City
Big dist in addr--------
Trattoria DellArte 900 7th Ave  between 56th and 57th Sts New York
Westside Cottage 689 Ninth Ave New York City
Big dist in cui--------
Cafeacute  RitzCarlton  Buckhead 3434 Peachtree Rd Atlanta
RitzCarlton Cafe Buckhead 3434 Peachtree Rd NE Atlanta
Big dist in cui--------
Cafeacute  RitzCarlton  Buckhead 3434 Peachtree Rd Atlanta
RitzCarlton Dining Room Buckhead 3434 Peachtree Rd NE Atlanta
Big dist in addr--------
Mifune Japan Center  Kintetsu Building  1737 Post St San Francisco
Mifune 1737 Post St San Francisco


In [18]:
# looks like using edit dist on cuisine is bad, therefore use normalized edit dist on addr only
np.histogram(addrDist)

(array([39, 25, 17, 16,  5,  7,  6,  1,  2,  2]),
 array([ 0.        ,  0.13636364,  0.27272727,  0.40909091,  0.54545455,
         0.68181818,  0.81818182,  0.95454545,  1.09090909,  1.22727273,
         1.36363636]))

In [19]:
# filter out those cands with addr dist more than 0.95
with open ("output.txt","w") as out:
    for i in range(len(addrDist)):
        if addrDist[i] < 0.95:
            out.write("zagats.csv:" + str(cand[1][i]) + "\t" + "fodors.csv:" + str(cand[0][i]) + "\n")

        